In [39]:
# ====== 0) 기본 세팅 ======
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lightgbm as lgb

In [40]:
# ====== 1) 데이터 로드 (상대경로 + 인코딩 fallback) ======
CANDIDATES = [
    Path("../../eda/data/merged_data.csv"),
    Path("./eda/data/merged_data.csv"),
    Path("eda/data/merged_data.csv"),
]

def load_csv_multi(paths):
    last_err = None
    for p in paths:
        if p.exists():
            for enc in ["utf-8-sig", "cp949", "utf-8"]:
                try:
                    return pd.read_csv(p, low_memory=False, encoding=enc)
                except Exception as e:
                    last_err = e
    raise FileNotFoundError(f"데이터 파일을 찾을 수 없습니다. 마지막 오류: {last_err}")

df = load_csv_multi(CANDIDATES)
print(df.shape)

(39975, 137)


In [41]:
# 필수 컬럼 체크
must_have = ["기준_년분기_코드", "자치구_코드_명", "서비스_업종_코드_명", "폐업_률"]
missing = [c for c in must_have if c not in df.columns]
if missing:
    raise ValueError(f"다음 컬럼이 필요합니다: {missing}")

In [42]:
# ====== 2) 타깃 생성: t 시점 피처로 t+1 분기의 '폐업_률' 예측 ======
# 그룹(자치구, 업종)별로 분기순 정렬 → 다음 분기의 폐업률을 y로 사용
df = df.sort_values(["자치구_코드_명", "서비스_업종_코드_명", "기준_년분기_코드"]).copy()
df["y"] = (
    df.groupby(["자치구_코드_명", "서비스_업종_코드_명"])["폐업_률"]
      .shift(-1)
)

# 마지막 분기(각 그룹의 말단)는 y가 비어있으므로 제거
df = df.dropna(subset=["y"]).reset_index(drop=True)
print("[INFO] After creating y (next-quarter 폐업_률):", df.shape)

[INFO] After creating y (next-quarter 폐업_률): (38419, 138)


In [43]:
# ====== 3) 피처 구성 (간단 버전) ======
# - 범주형: 자치구_코드_명, 서비스_업종_코드_명
# - 숫자형: 나머지(필요시 그대로 사용). t의 폐업_률(현재 분기)은 예측에 유용하므로 '피처'로 둠.
cat_cols = ["자치구_코드_명", "서비스_업종_코드_명"]
for c in cat_cols:
    if c in df.columns:
        df[c] = df[c].astype("category")

# 학습 입력 X / 타깃 y
# y와 누수 유발 보조 컬럼만 제외 (지금은 y만)
X = df.drop(columns=["y"])
y = df["y"]

In [44]:
# ===== 피처 구성 (순서형 처리 후 X/y 생성) =====
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

# 0) 기본 카테고리(문자) 컬럼
cat_cols = ["자치구_코드_명", "서비스_업종_코드_명"]

# 1) 상권_변화_지표: 순서형(약→강) + 수치 파생
biz_order = ["LL", "LH", "HL", "HH"]
biz_dtype = CategoricalDtype(categories=biz_order, ordered=True)

if "상권_변화_지표" in df.columns:
    col = "상권_변화_지표"
    # 문자열 정리
    df[col] = df[col].astype("string").str.strip()
    # 허용 외 라벨/빈값은 NaN
    df.loc[~df[col].isin(biz_order), col] = pd.NA
    # 순서형 카테고리로 변환
    df[col] = df[col].astype(biz_dtype)
    # 순위(0~3) 파생, 결측은 -1
    score_map = {"LL": 0, "LH": 1, "HL": 2, "HH": 3}
    df[col + "_순위"] = df[col].map(score_map).astype("float").fillna(-1.0)
    # 카테고리 목록에 원본 추가
    cat_cols.append(col)

# 2) 기본 카테고리 컬럼 캐스팅
for c in cat_cols:
    if c in df.columns:
        df[c] = df[c].astype("category")

# 3) 남아있는 object 컬럼 안전망(있다면 전부 카테고리로)
obj_cols = df.select_dtypes(include="object").columns
if len(obj_cols) > 0:
    df[obj_cols] = df[obj_cols].astype("category")

# 4) X, y 생성은 변환 "후"에!
X = df.drop(columns=["y"]).copy()
y = df["y"].copy()

# 5) LightGBM에 넘길 카테고리 목록
cat_in_X = [c for c in cat_cols if c in X.columns]

# 6) sanity check: X 안에 object가 남아있으면 바로 알려주기
bad_obj = X.select_dtypes(include="object").columns.tolist()
print("[CHECK] object columns in X:", bad_obj)  # [] 이어야 정상

[CHECK] object columns in X: []


In [45]:
# ====== 4) 시계열 분할: 최신 분기=테스트, 그 이전=검증(ES), 나머지=학습 ======
quarters = np.sort(df["기준_년분기_코드"].unique())
if len(quarters) < 3:
    # 분기가 3개 미만이면 단순 분할(80/20)로 fallback
    print("[WARN] 분기 수가 적어 시계열 3-way 분할이 어려워 80/20 랜덤 분할로 대체합니다.")
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    # 검증 세트는 학습 세트 일부로 만듦
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42
    )
else:
    test_q = quarters[-1]
    val_q  = quarters[-2]

    tr_mask  = X["기준_년분기_코드"] <  val_q
    val_mask = X["기준_년분기_코드"] == val_q
    te_mask  = X["기준_년분기_코드"] == test_q

    X_tr,  y_tr  = X[tr_mask],  y[tr_mask]
    X_val, y_val = X[val_mask], y[val_mask]
    X_test,y_test= X[te_mask],  y[te_mask]

print(f"[INFO] Train: {X_tr.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
print(f"[INFO] Test quarter = {int(test_q)} / Val quarter = {int(val_q) if len(quarters)>=2 else 'N/A'}")

[INFO] Train: (35359, 138), Val: (1531, 138), Test: (1529, 138)
[INFO] Test quarter = 20251 / Val quarter = 20244


In [46]:
# ====== 5) LightGBM 회귀 모델 ======
# 간단하고 안정적인 기본값 위주. (필요시 이후 튜닝)
model = lgb.LGBMRegressor(
    objective="regression",
    learning_rate=0.08,
    num_leaves=63,
    n_estimators=3000,          # ES가 있으니 넉넉히
    feature_fraction=0.9,
    bagging_fraction=0.9,
    bagging_freq=1,
    min_child_samples=50,
    reg_lambda=3.0,
    random_state=42,
    verbosity=-1,
)

# 카테고리 컬럼 전달 (존재하는 것만)
cat_in_X = [c for c in cat_cols if c in X.columns]

# 학습 (검증 세트로 EarlyStopping)
model.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    eval_metric="l2",                 # 회귀 기본(MSE). ES 기준으로 충분.
    categorical_feature=cat_in_X,
    callbacks=[lgb.early_stopping(100, verbose=False)]
)

LGBMRegressor(bagging_fraction=0.9, bagging_freq=1, feature_fraction=0.9,
              learning_rate=0.08, min_child_samples=50, n_estimators=3000,
              num_leaves=63, objective='regression', random_state=42,
              reg_lambda=3.0, verbosity=-1)

In [47]:
# ====== 6) 테스트 평가 ======
pred = model.predict(X_test)

mae  = mean_absolute_error(y_test, pred)
mse  = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
r2   = r2_score(y_test, pred)

print("\n========== Regression Metrics (Test) ==========")
print(f"MAE : {mae:,.6f}")
print(f"MSE : {mse:,.6f}")
print(f"RMSE: {rmse:,.6f}")
print(f"R²  : {r2:,.6f}")

# (옵션) 간단한 예측/실측 미리보기
preview = pd.DataFrame({
    "기준_년분기_코드": X_test["기준_년분기_코드"].values,
    "자치구_코드_명":   X_test["자치구_코드_명"].astype(str).values if "자치구_코드_명" in X_test else None,
    "서비스_업종_코드_명": X_test["서비스_업종_코드_명"].astype(str).values if "서비스_업종_코드_명" in X_test else None,
    "y_true(다음분기_폐업률)": y_test.values,
    "y_pred": pred
}).head(10)
print("\n[Preview] Test predictions:")
print(preview)


========== Regression Metrics (Test) ==========
MAE : 1.074429
MSE : 2.120283
RMSE: 1.456119
R²  : 0.354086

[Preview] Test predictions:
   기준_년분기_코드 자치구_코드_명 서비스_업종_코드_명  y_true(다음분기_폐업률)    y_pred
0      20251      강남구         PC방              10.7  5.051463
1      20251      강남구          가구               1.6  1.676396
2      20251      강남구          가방               0.6  2.147607
3      20251      강남구        가전제품               1.3  1.573574
4      20251      강남구      가전제품수리               2.5  1.664533
5      20251      강남구         고시원               0.0  5.607347
6      20251      강남구       골프연습장               3.7  2.401593
7      20251      강남구         네일숍               2.4  2.264024
8      20251      강남구         노래방               4.1  2.327907
9      20251      강남구         당구장               1.3  3.781649
